In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

## IMPORT CLEANED FILES

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'event_details.csv',
 'fighters.csv',
 'fighters_df.ipynb',
 'final_df.csv',
 'final_df_feature_engineering.ipynb',
 'matches.ipynb',
 'match_df.csv',
 'scraper',
 'Soupscraper.ipynb',
 'Untitled.ipynb']

In [4]:
UFC_files =[i for i in os.listdir() if i.endswith('.csv')]
print(UFC_files)

['event_details.csv', 'fighters.csv', 'final_df.csv', 'match_df.csv']


In [5]:
fighters = pd.read_csv('fighters.csv')
matches = pd.read_csv('match_df.csv')

In [6]:
fighters.head()

,name,nick_name,height,weight,reach,stance,date_birth,SLpm,StrAcc,SApm,StrDef,TDavg,TDacc,TDdef,SubAvg,Win,Lose,Draw
0,Mark Eddiva,NaN,68.0,145.0,68.0,Orthodox,1986-02-16,3.37,45.0,2.98,54.0,1.45,22.0,63.0,0.0,6,3,0
1,Scott Fiedler,NaN,76.0,235.0,NaN,Orthodox,NaN,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0,1,0
2,Drew Fickett,The Master,70.0,155.0,70.0,Orthodox,1979-12-14,2.33,41.0,2.29,57.0,1.30,35.0,30.0,2.2,42,21,0
3,Marcos Galvao,Loro,67.0,135.0,NaN,Orthodox,1981-06-23,1.97,32.0,3.86,61.0,2.27,28.0,88.0,1.1,18,8,1
4,Herdem Alacabek,NaN,74.0,205.0,75.0,Orthodox,1991-06-07,3.84,65.0,6.66,23.0,2.06,50.0,42.0,0.0,5,1,0


In [7]:
matches.head()

,Date,fight_name,fighter1_name,fighter2_name,kd_1,kd_2,sig_str_pct_1,sig_str_pct_2,td_pct_1,td_pct_2,sub_att_1,sub_att_2,pass_1,pass_2,sig_str_1_landed,sig_str_1_attempted,sig_str_2_landed,sig_str_2_attempted,total_str_1_landed,total_str_1_attempted,total_str_2_landed,total_str_2_attempted,td_1_landed,td_1_attempted,td_2_landed,td_2_attempted,head_1_landed,head_1_attempted,head_2_landed,head_2_attempted,body_1_landed,body_1_attempted,body_2_landed,body_2_attempted,leg_1_landed,leg_1_attempted,leg_2_landed,leg_2_attempted,distance_1_landed,distance_1_attempted,distance_2_landed,distance_2_attempted,clinch_1_landed,clinch_1_attempted,clinch_2_landed,clinch_2_attempted,ground_1_landed,ground_1_attempted,ground_2_landed,ground_2_attempted,City,State,Country,fighter1_outcome,fighter2_outcome,weight_bout,method,num_rounds,details,time,referee
0,2020-09-19,UFC Fight Night: Covington vs. Woodley,Tyson Nam,Jerome Rivera,1,0,0.50,0.37,0.0,0.00,0,0,0,0,36,72,30,80,36,72,31,81,0,0,0,0,27,58,9,45,7,12,2,11,2,2,19,24,18,47,29,79,3,3,1,1,15,22,0,0,Las Vegas,Nevada,USA,1,0,Bantamweight Bout,KO/TKO,2,Punches to Head On Ground,34,Chris Tognoni
1,2020-09-19,UFC Fight Night: Covington vs. Woodley,Andre Ewell,Irwin Rivera,0,0,0.41,0.46,0.0,0.11,0,0,1,1,123,299,81,176,124,301,85,180,0,0,1,9,78,226,49,134,36,64,14,24,9,9,18,18,119,293,74,164,4,6,1,4,0,0,6,8,Las Vegas,Nevada,USA,1,0,Bantamweight Bout,Decision - Split,3,NaN,300,Chris Tognoni
2,2020-09-19,UFC Fight Night: Covington vs. Woodley,Darrick Minner,TJ Laramie,0,0,0.88,0.00,0.0,0.00,1,0,0,0,15,17,0,4,18,20,0,4,0,0,0,1,1,3,0,3,13,13,0,1,1,1,0,0,2,4,0,3,13,13,0,1,0,0,0,0,Las Vegas,Nevada,USA,1,0,Featherweight Bout,Submission,1,Guillotine Choke From Guard,52,Dan Miragliotta
3,2020-09-19,UFC Fight Night: Covington vs. Woodley,Jessica-Rose Clark,Sarah Alpar,0,0,0.71,0.40,1.0,0.18,0,0,4,0,76,106,12,30,124,175,26,44,1,1,2,11,66,95,3,15,9,10,3,6,1,1,6,9,15,30,12,30,25,29,0,0,36,47,0,0,Las Vegas,Nevada,USA,1,0,Women's Bantamweight Bout,KO/TKO,3,Knee to Head In Clinch,261,Chris Tognoni
4,2020-09-19,UFC Fight Night: Covington vs. Woodley,Mayra Bueno Silva,Mara Romero Borella,0,0,0.28,0.66,0.0,0.50,1,0,0,0,2,7,6,9,3,8,15,21,0,0,1,2,1,5,5,7,0,1,0,1,1,1,1,1,2,7,4,7,0,0,0,0,0,0,2,2,Las Vegas,Nevada,USA,1,0,Women's Flyweight Bout,Submission,1,Armbar From Bottom Guard,149,Jason Herzog


In [8]:
#sort matches by date and reset the index
matches  = matches.sort_values('Date',ascending = True).reset_index(drop =True).copy()

#change date to datetime object
matches['Date'] = pd.to_datetime(matches['Date'])



# Feature Engineering

### CREATE DAYS SINCE LAST MATCH

In [9]:
# 98 draws, we will leave draws as is. A draw is when two fighters have 0 as their outcome
len(np.where(matches['fighter1_outcome'] == matches['fighter2_outcome'])[0])

98

In [10]:
names_melted = pd.melt(matches,id_vars= 'Date',value_vars= ['fighter1_name','fighter2_name'],var_name='fighter',value_name = 'name')

In [11]:
outcome_melted =\
pd.melt(matches, id_vars= 'Date', value_vars = ['fighter1_outcome','fighter2_outcome'],var_name='fighter',value_name = 'outcome')

In [12]:
#concatenate names and outcomes and drop the first 2 columns because they are duplicated
concated = pd.concat([names_melted,outcome_melted],axis=1).iloc[:,2:]

In [13]:
#sort by date
concated.sort_values('Date',inplace = True) 

In [14]:
concated['last_fight'] = concated.groupby(['name'])['Date'].diff().dt.days

### DAYS SINCE LAST WIN/ LOST

In [15]:
#find the wins
concated['days_since_win'] = concated['Date'].where(concated['outcome'].eq(1))
#find days between wins
concated['days_since_win'] =\
(concated['Date'].sub(concated.groupby(['name'])['days_since_win'].apply(lambda x:x.shift().ffill())).dt.days.fillna(0).astype(int))

In [16]:
#find the loses
concated['days_since_lose'] = concated['Date'].where(concated['outcome'].eq(0))
#find days between loses
concated['days_since_lose'] =\
(concated['Date'].sub(concated.groupby(['name'])['days_since_lose'].apply(lambda x:x.shift().ffill())).dt.days.fillna(0).astype(int))

In [17]:
"""Check to see if the variables are what they say they are"""
concated[concated['name'] == 'Chan Sung Jung']

,name,Date,fighter,outcome,last_fight,days_since_win,days_since_lose
1574,Chan Sung Jung,2011-03-26,fighter1_outcome,1,NaN,0,0
7530,Chan Sung Jung,2011-12-10,fighter2_outcome,1,259.0,259,0
1904,Chan Sung Jung,2012-05-15,fighter1_outcome,1,157.0,157,0
8088,Chan Sung Jung,2013-08-03,fighter2_outcome,0,445.0,445,0
9771,Chan Sung Jung,2017-02-04,fighter2_outcome,1,1281.0,1726,1281
4830,Chan Sung Jung,2018-11-10,fighter1_outcome,0,644.0,644,1925
10894,Chan Sung Jung,2019-06-22,fighter2_outcome,1,224.0,868,224
11174,Chan Sung Jung,2019-12-21,fighter2_outcome,1,182.0,182,406


In [18]:
#fill in any missing values for the days since last fight with 0
concated['last_fight'] = concated['last_fight'].fillna(value = 0)

In [19]:
concated.head()

,name,Date,fighter,outcome,last_fight,days_since_win,days_since_lose
0,Royce Gracie,1993-11-12,fighter1_outcome,1,0.0,0,0
5757,Zane Frazier,1993-11-12,fighter2_outcome,0,0.0,0,0
5758,Art Jimmerson,1993-11-12,fighter2_outcome,0,0.0,0,0
5754,Teila Tuli,1993-11-12,fighter2_outcome,0,0.0,0,0
5751,Ken Shamrock,1993-11-12,fighter2_outcome,0,0.0,0,0


### Add the new variables to matches
* Change the df to long to wide inorder to concat with the df "matches"
* Concat the new concatenated df with matches

In [20]:
concated.shape[0]/2

5751.0

In [21]:
#split concated in half after sorting the index
fighter1 = concated.sort_index()[:5751]
fighter2 = concated.sort_index()[5751:]

In [22]:
fighter1_rename = {'name':'fighter1_name', 'outcome':'fighter1_outcome',\
                   'last_fight':'fighter1_last_fight','days_since_win':'fighter1_since_win',\
                   'days_since_lose':'fighter1_since_lose'}
fighter1.rename(fighter1_rename,axis=1,inplace = True)
fighter1.drop(['Date','fighter'],axis=1, inplace= True)

In [23]:
fighter2_rename = {'name':'fighter2_name', 'outcome':'fighter2_outcome',\
                   'last_fight':'fighter2_last_fight','days_since_win':'fighter2_since_win',\
                   'days_since_lose':'fighter2_since_lose'}
fighter2.rename(fighter2_rename,axis=1,inplace = True)
fighter2.drop(['Date','fighter'],axis=1, inplace= True)
fighter2.reset_index(drop = True,inplace =True)

In [24]:
fighters_days_since = pd.concat([fighter1,fighter2],axis=1)

In [25]:
fighters_days_since.head()

,fighter1_name,fighter1_outcome,fighter1_last_fight,fighter1_since_win,fighter1_since_lose,fighter2_name,fighter2_outcome,fighter2_last_fight,fighter2_since_win,fighter2_since_lose
0,Royce Gracie,1,0.0,0,0,Ken Shamrock,0,0.0,0,0
1,Royce Gracie,1,0.0,0,0,Gerard Gordeau,0,0.0,0,0
2,Jason DeLucia,1,0.0,0,0,Trent Jenkins,0,0.0,0,0
3,Gerard Gordeau,1,0.0,0,0,Teila Tuli,0,0.0,0,0
4,Ken Shamrock,1,0.0,0,0,Patrick Smith,0,0.0,0,0


In [26]:
#merge
matches2 = matches.merge(fighters_days_since,left_index= True,right_index=True,suffixes=('','_y')).copy()
#drop the duplicated column names
matches2.drop(['fighter1_name_y','fighter1_outcome_y','fighter2_name_y','fighter2_outcome_y'],axis=1,inplace = True)

## MERGING fighters and matches
* Merge fighters and matches
* Export as CSV

In [27]:
fighters.shape

(3343, 18)

In [28]:
matches2.shape

(5751, 67)

In [29]:
#merge on the fighter stats on fighter1, and then merge again on fighter 2
df =\
matches2.merge(fighters, left_on = 'fighter1_name',right_on = 'name')\
.merge(fighters,left_on ='fighter2_name',right_on = 'name', suffixes = ('_1','_2'))


In [30]:
#drop name_1 and name_2 because we already have fighter1 and fighter2
df.drop(['name_1','name_2'],axis = 1, inplace = True)

In [31]:
df.shape

(5754, 101)

In [32]:
df.sample(3)

,Date,fight_name,fighter1_name,fighter2_name,kd_1,kd_2,sig_str_pct_1,sig_str_pct_2,td_pct_1,td_pct_2,sub_att_1,sub_att_2,pass_1,pass_2,sig_str_1_landed,sig_str_1_attempted,sig_str_2_landed,sig_str_2_attempted,total_str_1_landed,total_str_1_attempted,total_str_2_landed,total_str_2_attempted,td_1_landed,td_1_attempted,td_2_landed,td_2_attempted,head_1_landed,head_1_attempted,head_2_landed,head_2_attempted,body_1_landed,body_1_attempted,body_2_landed,body_2_attempted,leg_1_landed,leg_1_attempted,leg_2_landed,leg_2_attempted,distance_1_landed,distance_1_attempted,distance_2_landed,distance_2_attempted,clinch_1_landed,clinch_1_attempted,clinch_2_landed,clinch_2_attempted,ground_1_landed,ground_1_attempted,ground_2_landed,ground_2_attempted,City,State,Country,fighter1_outcome,fighter2_outcome,weight_bout,method,num_rounds,details,time,referee,fighter1_last_fight,fighter1_since_win,fighter1_since_lose,fighter2_last_fight,fighter2_since_win,fighter2_since_lose,nick_name_1,height_1,weight_1,reach_1,stance_1,date_birth_1,SLpm_1,StrAcc_1,SApm_1,StrDef_1,TDavg_1,TDacc_1,TDdef_1,SubAvg_1,Win_1,Lose_1,Draw_1,nick_name_2,height_2,weight_2,reach_2,stance_2,date_birth_2,SLpm_2,StrAcc_2,SApm_2,StrDef_2,TDavg_2,TDacc_2,TDdef_2,SubAvg_2,Win_2,Lose_2,Draw_2
1713,2016-02-27,UFC Fight Night: Silva vs Bisping,Gegard Mousasi,Thales Leites,0,0,0.47,0.24,0.0,0.00,0,0,1,0,81,171,17,70,95,187,19,72,0,0,0,10,55,141,12,60,11,11,0,3,15,19,5,7,66,154,17,69,9,9,0,1,6,8,0,0,London,England,United Kingdom,1,0,Middleweight Bout,Decision - Unanimous,3,NaN,300,Marc Goddard,154.0,287,154,224.0,392,224,NaN,74.0,185.0,76.0,Orthodox,1985-08-01,3.75,50.0,1.21,68.0,1.59,60.0,59.0,1.1,42,6,2,NaN,73.0,185.0,78.0,Orthodox,1981-09-06,2.13,40.0,2.99,52.0,2.10,27.0,30.0,1.1,28,9,0
925,2010-10-16,UFC 120: Bisping vs Akiyama,Cyrille Diabate,Alexander Gustafsson,0,1,0.50,0.50,0.0,0.66,0,3,0,7,11,22,28,56,13,24,60,94,0,1,2,3,6,14,23,46,3,5,2,4,2,3,3,6,10,17,7,25,1,5,7,11,0,0,14,20,London,England,United Kingdom,0,1,Light Heavyweight Bout,Submission,2,Rear Naked Choke,161,Leon Roberts,140.0,140,0,189.0,336,189,The Snake,78.0,205.0,81.0,Southpaw,1973-10-06,4.06,71.0,1.78,60.0,1.17,83.0,53.0,0.2,19,10,1,The Mauler,77.0,205.0,79.0,Orthodox,1987-01-15,4.02,40.0,3.42,51.0,1.54,39.0,83.0,0.4,18,7,0
3264,2020-09-19,UFC Fight Night: Covington vs. Woodley,Johnny Walker,Ryan Spann,0,2,0.86,0.53,0.0,1.00,0,0,0,1,26,30,7,13,30,34,10,16,0,0,2,2,24,27,7,13,1,2,0,0,1,1,0,0,4,5,3,7,0,0,0,0,22,25,4,6,Las Vegas,Nevada,USA,1,0,Light Heavyweight Bout,KO/TKO,1,Punches to Head On Ground,163,Dan Miragliotta,189.0,567,189,133.0,133,0,NaN,78.0,205.0,82.0,Orthodox,1992-03-30,4.41,72.0,3.51,31.0,0.40,100.0,60.0,1.2,18,5,0,Superman,77.0,205.0,79.0,Orthodox,1991-08-24,3.27,44.0,3.43,46.0,1.76,41.0,60.0,1.8,18,6,0


## CREATE age during fight
* set dob as datetimeindex
* subtract event date by dob

In [33]:
df.sort_values('Date',inplace = True)

In [34]:
df['date_birth_1'] = pd.DatetimeIndex(df['date_birth_1'])
df['date_birth_2'] = pd.DatetimeIndex(df['date_birth_2'])

In [35]:
df['age_1'] = df['Date'].dt.year - df['date_birth_1'].dt.year
df['age_2'] = df['Date'].dt.year - df['date_birth_2'].dt.year

## UNIQUE STATS FOR EACH FIGHT
* Create unique stats for each fight for each fighter
* Get the cumulative sum for each stat
* Get the average by dividing actual/attempts

In [39]:
df.columns

Index(['Date', 'fight_name', 'fighter1_name', 'fighter2_name', 'kd_1', 'kd_2',
       'sig_str_pct_1', 'sig_str_pct_2', 'td_pct_1', 'td_pct_2',
       ...
       'StrDef_2', 'TDavg_2', 'TDacc_2', 'TDdef_2', 'SubAvg_2', 'Win_2',
       'Lose_2', 'Draw_2', 'age_1', 'age_2'],
      dtype='object', length=103)

In [40]:
"""Separate the dataframe to 2 different dataframes. 1 for each fighter.
    Because a fighter can be fighter_1 on one day and fighter_2 a different day
"""
#True if column name contains 1
col_names1 = ['1' in i for i in df.columns] 
#True if column name contains 2
col_names2 = ['2' in i for i in df.columns]

#Map the column names with values from col_names(x)
col_names1 = dict(zip(df.columns,col_names1))
col_names2 = dict(zip(df.columns,col_names2))

#create empty list to append column names for the stats for fighter1 & fighter2
stats_1 = []
stats_2 = []

#get stats for fighter1 in stats_1 and get stats for fighter2 in stats_2
for i,v in col_names1.items():
    if v == True:
        stats_1.append(i)
for i,v in col_names2.items():
    if v == True:
        stats_2.append(i)

In [41]:
#insert date to get matching column names
stats_1.insert(0,'Date')
stats_2.insert(0,'Date')

In [42]:
mapper = dict(zip(df[stats_2].columns,df[stats_1].columns))

In [43]:
#df[stats_2].rename(mapper,axis=1)
long = pd.concat([df[stats_1],df[stats_2].rename(mapper,axis=1)])
long_sorted = long.reset_index().sort_values('Date')

In [58]:
long_sorted_indexed = long_sorted.iloc[:,:26].copy()

In [59]:
long_sorted.shape

(11508, 48)

In [60]:
"""create a function to get cumulative stats for each fighter.
    Divide landed by attemped to get a cumulative accuracy for each fight.  
    A fighters stats change over time. 
    Every match will have different stats according to the fighters previous cumulative performance.
"""
def cumulative_stats_avg(df,column_landed,column_attempted,fighter_number ='fighter1_name'):
    a = df.groupby([fighter_number])[column_landed].apply(lambda x: x.shift(1).cumsum())
    b = df.groupby([fighter_number])[column_attempted].apply(lambda x: x.shift(1).cumsum()) 
    c = a/b
    return a,b,c

In [61]:
long_sorted_indexed['sig_str_landed'], long_sorted_indexed['sig_str_attempted'], long_sorted_indexed['sig_str_freq'] =\
                                        cumulative_stats_avg(long_sorted_indexed,"sig_str_1_landed","sig_str_1_attempted" )

long_sorted_indexed['total_str_landed'], long_sorted_indexed['total_str_attempted'], long_sorted_indexed['total_str_freq'] =\
                                        cumulative_stats_avg(long_sorted_indexed,'total_str_1_landed','total_str_1_attempted' )

long_sorted_indexed['td_landed'], long_sorted_indexed['td_attempted'], long_sorted_indexed['td_freq'] =\
                                        cumulative_stats_avg(long_sorted_indexed,'td_1_landed','td_1_attempted')

long_sorted_indexed['head_landed'], long_sorted_indexed['head_attempted'], long_sorted_indexed['head_freq'] =\
                                        cumulative_stats_avg(long_sorted_indexed,'head_1_landed', 'head_1_attempted')

long_sorted_indexed['body_landed'], long_sorted_indexed['body_attempted'], long_sorted_indexed['body_freq'] =\
                                        cumulative_stats_avg(long_sorted_indexed,'body_1_landed','body_1_attempted')

long_sorted_indexed['leg_landed'], long_sorted_indexed['leg_attempted'], long_sorted_indexed['leg_freq'] =\
                                        cumulative_stats_avg(long_sorted_indexed,'leg_1_landed', 'leg_1_attempted')

long_sorted_indexed['distance_landed'], long_sorted_indexed['distance_attempted'], long_sorted_indexed['distance_freq'] =\
                                        cumulative_stats_avg(long_sorted_indexed, 'distance_1_landed', 'distance_1_attempted')

long_sorted_indexed['clinch_landed'], long_sorted_indexed['clinch_attempted'], long_sorted_indexed['clinch_freq'] =\
                                        cumulative_stats_avg(long_sorted_indexed, 'clinch_1_landed','clinch_1_attempted')

long_sorted_indexed['ground_landed'], long_sorted_indexed['ground_attempted'], long_sorted_indexed['ground_freq'] =\
                                        cumulative_stats_avg(long_sorted_indexed,'ground_1_landed', 'ground_1_attempted')

In [104]:
""" BACK TO WIDE FORM"""

' BACK TO WIDE FORM'

In [62]:
fighter_1 = long_sorted_indexed.sort_index().loc[:5753]
fighter_2 = long_sorted_indexed.sort_index().loc[5754:]

In [64]:
#rename fighter_2 so it is different from fighter_1
#replace 1 with ''
#add the suffix '2'

import re
fighter_2_col_name = []

for i in fighter_2.columns:
    i = re.sub('[1]','',i)
    fighter_2_col_name.append(i)

#rename the fighter_2 with the 1 removed    
fighter_2.columns = fighter_2_col_name
#add the suffix '2' to column  name
fighter_2 = fighter_2.add_suffix('2')

In [69]:
#merge the 2 fighter columns
fighters_merged= pd.merge(fighter_1,fighter_2,left_on ='index',right_on='index2')

In [116]:
#merge the fighters with the original df
final_df = df.merge(fighters_merged,left_on = df.index,right_on='index',suffixes=('', '_y')).copy()

In [117]:
final_df =final_df.T.drop_duplicates().T

In [118]:
final_df = final_df.drop('index',axis=1).copy()

In [119]:
#make final_df in current directory
final_df.to_csv('final_df.csv',index=False)

In [121]:
final_df[final_df['fighter1_name'] == 'Chan Sung Jung']

,Date,fight_name,fighter1_name,fighter2_name,kd_1,kd_2,sig_str_pct_1,sig_str_pct_2,td_pct_1,td_pct_2,sub_att_1,sub_att_2,pass_1,pass_2,sig_str_1_landed,sig_str_1_attempted,sig_str_2_landed,sig_str_2_attempted,total_str_1_landed,total_str_1_attempted,total_str_2_landed,total_str_2_attempted,td_1_landed,td_1_attempted,td_2_landed,td_2_attempted,head_1_landed,head_1_attempted,head_2_landed,head_2_attempted,body_1_landed,body_1_attempted,body_2_landed,body_2_attempted,leg_1_landed,leg_1_attempted,leg_2_landed,leg_2_attempted,distance_1_landed,distance_1_attempted,distance_2_landed,distance_2_attempted,clinch_1_landed,clinch_1_attempted,clinch_2_landed,clinch_2_attempted,ground_1_landed,ground_1_attempted,ground_2_landed,ground_2_attempted,City,State,Country,fighter1_outcome,fighter2_outcome,weight_bout,method,num_rounds,details,time,referee,fighter1_last_fight,fighter1_since_win,fighter1_since_lose,fighter2_last_fight,fighter2_since_win,fighter2_since_lose,nick_name_1,height_1,weight_1,reach_1,stance_1,date_birth_1,SLpm_1,StrAcc_1,SApm_1,StrDef_1,TDavg_1,TDacc_1,TDdef_1,SubAvg_1,Win_1,Lose_1,Draw_1,nick_name_2,height_2,weight_2,reach_2,stance_2,date_birth_2,SLpm_2,StrAcc_2,SApm_2,StrDef_2,TDavg_2,TDacc_2,TDdef_2,SubAvg_2,Win_2,Lose_2,Draw_2,age_1,age_2,sig_str_landed,sig_str_attempted,sig_str_freq,total_str_landed,total_str_attempted,total_str_freq,td_landed,td_attempted,td_freq,head_landed,head_attempted,head_freq,body_landed,body_attempted,body_freq,leg_landed,leg_attempted,leg_freq,distance_landed,distance_attempted,distance_freq,clinch_landed,clinch_attempted,clinch_freq,ground_landed,ground_attempted,ground_freq,sig_str_landed2,sig_str_attempted2,sig_str_freq2,total_str_landed2,total_str_attempted2,total_str_freq2,td_landed2,td_attempted2,td_freq2,head_landed2,head_attempted2,head_freq2,body_landed2,body_attempted2,body_freq2,leg_landed2,leg_attempted2,leg_freq2,distance_landed2,distance_attempted2,distance_freq2,clinch_landed2,clinch_attempted2,clinch_freq2,ground_landed2,ground_attempted2,ground_freq2
1554,2011-03-26,UFC Fight Night: Nogueira vs Davis,Chan Sung Jung,Leonard Garcia,0,0,0.3,0.17,0.5,0,1,0,4,0,30,100,15,87,47,119,16,88,1,2,0,1,22,90,10,75,2,2,2,3,6,8,3,9,20,84,13,83,2,2,2,4,8,14,0,0,Seattle,Washington,USA,1,0,NaN,Submission,2,Other - Lock On Ground\n Twister,299,Dan Miragliotta,0,0,0,112,112,1284,The Korean Zombie,67,145,72,Orthodox,1987-03-17,4.58,0.41,3.73,0.6,0.78,0.41,0.77,0.6,16,5,0,Bad Boy,70,145,68,Orthodox,1979-07-14,2.77,0.24,4.79,0.52,0.3,0.26,0.42,0.5,18,13,1,24,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113,409,0.276284,234,556,0.420863,2,5,0.4,77,351,0.219373,21,35,0.6,15,23,0.652174,82,357,0.229692,15,28,0.535714,16,24,0.666667
1902,2012-05-15,UFC on FUEL TV: Korean Zombie vs Poirier,Chan Sung Jung,Dustin Poirier,0,0,0.48,0.44,1,0,3,0,3,1,74,154,56,127,149,240,70,146,4,4,0,4,59,136,44,111,12,15,1,2,3,3,11,14,60,135,55,124,5,6,1,3,9,13,0,0,Fairfax,Virginia,USA,1,0,NaN,Submission,4,D'Arce Choke From Front Headlock\n Techni...,67,Dan Miragliotta,157,157,0,101,101,0,The Korean Zombie,67,145,72,Orthodox,1987-03-17,4.58,0.41,3.73,0.6,0.78,0.41,0.77,0.6,16,5,0,The Diamond,69,155,72,Southpaw,1989-01-19,5.57,0.5,4.18,0.54,1.46,0.35,0.6,1.3,26,6,0,25,23,36,110,0.327273,53,129,0.410853,1,2,0.5,28,100,0.28,2,2,1,6,8,0.75,21,86,0.244186,2,2,1,13,22,0.590909,182,366,0.497268,236,427,0.552693,7,13,0.538462,131,303,0.432343,18,26,0.692308,33,37,0.891892,115,237,0.485232,53,98,0.540816,14,31,0.451613
4827,2018-11-10,UFC Fight Night: Korean Zombie vs. Rodriguez,Chan Sung Jung,Yair Rodriguez,0,1,0.4,0.43,0,0,0,0,0,0,126,308,119,271,130,312,129,281,0,5,0,4,111,285,59,190,9,14,24,38,6,9,36,43,114,293,113,262,12,15,6,9,0,0,0,0,Denver,Colorado,USA,0,1,NaN,KO/TKO,5,Elbow to Head At Distance,299,Kevin MacDonald,644,644,1925,546,664,546,The Korean Zombie,67,145,72,Orthodox,1987-03-17,4.58,0.41,3.73,0.6,0.78,0.41,0.77,0.6,16,5,0,El Pantera,71,145,71,Orthod

In [122]:
final_df[final_df['fighter2_name'] == 'Chan Sung Jung']

,Date,fight_name,fighter1_name,fighter2_name,kd_1,kd_2,sig_str_pct_1,sig_str_pct_2,td_pct_1,td_pct_2,sub_att_1,sub_att_2,pass_1,pass_2,sig_str_1_landed,sig_str_1_attempted,sig_str_2_landed,sig_str_2_attempted,total_str_1_landed,total_str_1_attempted,total_str_2_landed,total_str_2_attempted,td_1_landed,td_1_attempted,td_2_landed,td_2_attempted,head_1_landed,head_1_attempted,head_2_landed,head_2_attempted,body_1_landed,body_1_attempted,body_2_landed,body_2_attempted,leg_1_landed,leg_1_attempted,leg_2_landed,leg_2_attempted,distance_1_landed,distance_1_attempted,distance_2_landed,distance_2_attempted,clinch_1_landed,clinch_1_attempted,clinch_2_landed,clinch_2_attempted,ground_1_landed,ground_1_attempted,ground_2_landed,ground_2_attempted,City,State,Country,fighter1_outcome,fighter2_outcome,weight_bout,method,num_rounds,details,time,referee,fighter1_last_fight,fighter1_since_win,fighter1_since_lose,fighter2_last_fight,fighter2_since_win,fighter2_since_lose,nick_name_1,height_1,weight_1,reach_1,stance_1,date_birth_1,SLpm_1,StrAcc_1,SApm_1,StrDef_1,TDavg_1,TDacc_1,TDdef_1,SubAvg_1,Win_1,Lose_1,Draw_1,nick_name_2,height_2,weight_2,reach_2,stance_2,date_birth_2,SLpm_2,StrAcc_2,SApm_2,StrDef_2,TDavg_2,TDacc_2,TDdef_2,SubAvg_2,Win_2,Lose_2,Draw_2,age_1,age_2,sig_str_landed,sig_str_attempted,sig_str_freq,total_str_landed,total_str_attempted,total_str_freq,td_landed,td_attempted,td_freq,head_landed,head_attempted,head_freq,body_landed,body_attempted,body_freq,leg_landed,leg_attempted,leg_freq,distance_landed,distance_attempted,distance_freq,clinch_landed,clinch_attempted,clinch_freq,ground_landed,ground_attempted,ground_freq,sig_str_landed2,sig_str_attempted2,sig_str_freq2,total_str_landed2,total_str_attempted2,total_str_freq2,td_landed2,td_attempted2,td_freq2,head_landed2,head_attempted2,head_freq2,body_landed2,body_attempted2,body_freq2,leg_landed2,leg_attempted2,leg_freq2,distance_landed2,distance_attempted2,distance_freq2,clinch_landed2,clinch_attempted2,clinch_freq2,ground_landed2,ground_attempted2,ground_freq2
1775,2011-12-10,UFC 140: Jones vs Machida,Mark Hominick,Chan Sung Jung,0,1,0,0.6,0,0,0,0,0,0,0,1,6,10,0,1,6,10,0,0,0,0,0,1,6,10,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,0,0,5,8,Toronto,Ontario,Canada,0,1,NaN,KO/TKO,1,Punch to Head At Distance,7,Herb Dean,224,322,224,259,259,0,The Machine,68,145,68,Orthodox,1982-07-22,4.08,0.42,4.18,0.69,0.25,0.66,0.71,0.9,20,12,0,The Korean Zombie,67,145,72,Orthodox,1987-03-17,4.58,0.41,3.73,0.6,0.78,0.41,0.77,0.6,16,5,0,29,24,145,413,0.35109,230,515,0.446602,1,2,0.5,97,346,0.280347,38,52,0.730769,10,15,0.666667,112,367,0.305177,12,14,0.857143,21,32,0.65625,30,100,0.3,47,119,0.394958,1,2,0.5,22,90,0.244444,2,2,1,6,8,0.75,20,84,0.238095,2,2,1,8,14,0.571429
2336,2013-08-03,UFC 163: Aldo vs Jung,Jose Aldo,Chan Sung Jung,0,0,0.36,0.14,0.83,0,0,0,2,0,32,87,17,117,43,99,42,145,5,6,0,0,24,77,13,106,7,8,2,5,1,2,2,6,28,78,14,112,0,0,3,4,4,9,0,1,Rio de Janeiro,Brazil,Brazil,1,0,NaN,KO/TKO,4,Punches to Head On Ground,120,Herb Dean,182,182,0,445,445,0,NaN,67,135,70,Orthodox,1986-09-09,3.48,0.44,3.55,0.62,0.6,0.61,0.9,0.1,28,7,0,The Korean Zombie,67,145,72,Orthodox,1987-03-17,4.58,0.41,3.73,0.6,0.78,0.41,0.77,0.6,16,5,0,27,26,232,606,0.382838,288,669,0.430493,5,9,0.555556,181,533,0.339587,12,24,0.5,39,49,0.795918,186,531,0.350282,9,15,0.6,37,60,0.616667,110,264,0.416667,202,369,0.547425,5,6,0.833333,87,236,0.368644,14,17,0.823529,9,11,0.818182,81,221,0.366516,7,8,0.875,22,35,0.628571
4021,2017-02-04,UFC Fight Night: Bermudez vs. The Korean Zombie,Dennis Bermudez,Chan Sung Jung,0,1,0.58,0.48,0,0,0,0,0,0,20,34,17,35,24,38,19,37,0,3,0,1,11,24,16,32,6,7,1,2,3,3,0,1,14,26,12,30,6,8,2,2,0,0,3,3,Houston,Texas,USA,0,1,NaN,KO/TKO,1,Punch to Head At Distance,149,Herb Dean,182,182,574,1281,1726,1281,The Menace,66,155,66,Orthodox,1986-12-13,4.42,0.46,3.07,0.59,3.89,0.4,0.82,1.1,17,9,0,The Korean Zombie,67,145,72,Orthodox,1987-03-17,4.58,0.41,3.73,0.6,0.78,0.41,0.77,0.6,16,5,0,31,30,573,1255,0.456574,933,1665,0.56036,38,84,0.45

In [135]:
final_df.drop(final_df.iloc[:,14:50],axis=1)

,Date,fight_name,fighter1_name,fighter2_name,kd_1,kd_2,sig_str_pct_1,sig_str_pct_2,td_pct_1,td_pct_2,sub_att_1,sub_att_2,pass_1,pass_2,City,State,Country,fighter1_outcome,fighter2_outcome,weight_bout,method,num_rounds,details,time,referee,fighter1_last_fight,fighter1_since_win,fighter1_since_lose,fighter2_last_fight,fighter2_since_win,fighter2_since_lose,nick_name_1,height_1,weight_1,reach_1,stance_1,date_birth_1,SLpm_1,StrAcc_1,SApm_1,StrDef_1,TDavg_1,TDacc_1,TDdef_1,SubAvg_1,Win_1,Lose_1,Draw_1,nick_name_2,height_2,weight_2,reach_2,stance_2,date_birth_2,SLpm_2,StrAcc_2,SApm_2,StrDef_2,TDavg_2,TDacc_2,TDdef_2,SubAvg_2,Win_2,Lose_2,Draw_2,age_1,age_2,sig_str_landed,sig_str_attempted,sig_str_freq,total_str_landed,total_str_attempted,total_str_freq,td_landed,td_attempted,td_freq,head_landed,head_attempted,head_freq,body_landed,body_attempted,body_freq,leg_landed,leg_attempted,leg_freq,distance_landed,distance_attempted,distance_freq,clinch_landed,clinch_attempted,clinch_freq,ground_landed,ground_attempted,ground_freq,sig_str_landed2,sig_str_attempted2,sig_str_freq2,total_str_landed2,total_str_attempted2,total_str_freq2,td_landed2,td_attempted2,td_freq2,head_landed2,head_attempted2,head_freq2,body_landed2,body_attempted2,body_freq2,leg_landed2,leg_attempted2,leg_freq2,distance_landed2,distance_attempted2,distance_freq2,clinch_landed2,clinch_attempted2,clinch_freq2,ground_landed2,ground_attempted2,ground_freq2
0,1993-11-12,UFC 1: The Beginning,Royce Gracie,Ken Shamrock,0,0,0,0,0,0,1,0,2,0,Denver,Colorado,USA,1,0,Open Weight Bout,Submission,1,Rear Naked Choke,57,Joao Alberto Barreto,0,0,0,0,0,0,NaN,73,175,NaN,Southpaw,1966-12-12,0.88,0.41,1.13,0.37,0,0,0.66,0.8,15,2,3,The World's Most Dangerous Man,73,205,72,Orthodox,1964-02-11,1.47,0.46,4.3,0.39,0.94,0.6,0.44,2.5,28,17,2,27,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-11-12,UFC 1: The Beginning,Royce Gracie,Gerard Gordeau,0,0,0.5,0,0.33,0,1,0,2,0,Denver,Colorado,USA,1,0,NaN,Submission,1,Rear Naked Choke,104,Joao Alberto Barreto,0,0,0,0,0,0,NaN,73,175,NaN,Southpaw,1966-12-12,0.88,0.41,1.13,0.37,0,0,0.66,0.8,15,2,3,NaN,77,216,NaN,Orthodox,1959-03-30,0,0,0,0,0,0,0,0,2,2,0,27,34,0,3,0,16,19,0.842105,1,2,0.5,0,1,0,0,0,NaN,0,2,0,0,3,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-11-12,UFC 1: The Beginning,Royce Gracie,Art Jimmerson,0,0,0,0,1,0,0,0,2,0,Denver,Colorado,USA,1,0,Open Weight Bout,Submission,1,Other \n Position - Mount,138,Joao Alberto Barreto,0,0,0,0,0,0,NaN,73,175,NaN,Southpaw,1966-12-12,0.88,0.41,1.13,0.37,0,0,0.66,0.8,15,2,3,NaN,73,196,NaN,Orthodox,1963-08-04,0,0,0,0,0,0,0,0,0,1,0,27,30,0,0,NaN,12,12,1,0,1,0,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-11-12,UFC 1: The Beginning,Kevin Rosier,Zane Frazier,2,0,0.55,0.42,0,0,0,0,0,0,Denver,Colorado,USA,1,0,Open Weight Bout,KO/TKO,1,to \n Corner Stoppage,260,Joao Alberto Barreto,0,0,0,0,0,0,NaN,76,275,NaN,Orthodox,NaT,0,0,0,0,0,0,0,0,2,6,0,Nasty,77,250,NaN,Orthodox,NaT,0,0,0,0,0,0,0,0,4,11,0,NaN,NaN,0,3,0,0,3,0,0,0,NaN,0,1,0,0,1,0,0,1,0,0,3,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-11-12,UFC 1: The Beginning,Ken Shamrock,Patrick Smith,0,0,1,0.5,0.5,0,2,0,0,0,Denver,Colorado,USA,1,0,Open Weight Bout,Submission,1,Heel Hook From Guard,109,Joao Alberto Barreto,0,0,0,0,0,0,The World's Most Dangerous Man,73,205,72,Orthodox,1964-02-11,1.47,0.46,4.3,0.39,0.94,0.6,0.44,2.5,28,17,2,NaN,74,225,NaN,Orthodox,1963-08-28,0,0,0,0,0,0,0,0,20,17,0,29,30,0,0,NaN,0,0,NaN,0,1,0,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN